In [1]:
# CELL 1: Clean Room Installation (with Fixed Gradio)

import os

# 1. Create constraints file
with open("constraints.txt", "w") as f:
    f.write("numpy<2.0\n")
    f.write("huggingface_hub<0.25.0\n")

# 2. Uninstall potential conflicts
print("🧹 Purging conflicts...")
os.system("pip uninstall -y tensorflow tensorflow-cpu tensorflow-gpu protobuf numpy gradio gradio_client -q")

# 3. Install stack with PINNED Gradio version
print("📦 Installing Stack...")
cmd = (
    "pip install "
    "numpy==1.26.4 "
    "diffusers==0.26.3 "
    "transformers==4.38.2 "
    "accelerate==0.27.2 "
    "peft==0.9.0 "
    "ftfy "
    "tensorboard "
    "-c constraints.txt -q"
)
os.system(cmd)

# Uninstall Broken bitsandbytes
import os
print("🚑 Removing broken bitsandbytes library...")
# We uninstall it so peft stops trying to import it and crashing.
os.system("pip uninstall -y bitsandbytes")
print("✅ Broken library removed. Falling back to standard precision.")
import numpy
print(f"✅ NumPy Version: {numpy.__version__}")

print("✅ All dependencies installed!")

🧹 Purging conflicts...


📦 Installing Stack...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 137.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires tensorflow>=2.2.0, which is not installed.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 6.33.2 which is incompatible.
datasets 4.4.1 requires huggingface-hub<2.0,>=0.25.0, but you have huggingface-hub 0.24.7 which is incompatible.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible

🚑 Removing broken bitsandbytes library...
✅ Broken library removed. Falling back to standard precision.
✅ NumPy Version: 1.26.4
✅ All dependencies installed!


In [20]:
# CELL 2: Load Model with Fashion-Gen Dataset Support (H5 Format)
import torch
import cv2
import numpy as np
from PIL import Image
import os
import glob
import h5py
import tempfile
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler

# --- 1. Find LoRA Weights ---
print("🔍 Searching for LoRA weights...")
LORA_WEIGHTS_PATH = None

for folder in os.listdir("/kaggle/input"):
    # Check for direct file in folder (your structure)
    direct_path = f"/kaggle/input/{folder}"
    weights_file = os.path. join(direct_path, "pytorch_lora_weights.safetensors")
    if os.path.exists(weights_file):
        LORA_WEIGHTS_PATH = direct_path
        print(f"✅ Found LoRA weights at: {LORA_WEIGHTS_PATH}")
        break
    
    # Also check subfolder structure (original structure)
    subfolder_path = f"/kaggle/input/{folder}/fashion_lora_output"
    weights_file_sub = os.path. join(subfolder_path, "pytorch_lora_weights. safetensors")
    if os. path.exists(weights_file_sub):
        LORA_WEIGHTS_PATH = subfolder_path
        print(f"✅ Found LoRA weights at: {LORA_WEIGHTS_PATH}")
        break

if LORA_WEIGHTS_PATH is None:  
    raise FileNotFoundError("❌ Could not find LoRA weights.  Please add your dataset as input.")

# --- 2. Extract Images from Fashion-Gen H5 Dataset ---
print("🔍 Searching for Fashion-Gen H5 dataset...")
H5_FILE_PATH = None
sample_image_paths = {}

# Search for H5 files
for folder in os. listdir("/kaggle/input"):
    folder_path = f"/kaggle/input/{folder}"
    h5_files = glob.glob(os.path.join(folder_path, "*.h5"))
    
    if h5_files: 
        # Prefer validation file (smaller), then train
        for h5_file in h5_files: 
            if 'validation' in h5_file. lower():
                H5_FILE_PATH = h5_file
                break
        if H5_FILE_PATH is None:
            H5_FILE_PATH = h5_files[0]
        
        print(f"✅ Found Fashion-Gen H5 file:  {H5_FILE_PATH}")
        break

if H5_FILE_PATH is None:
    raise FileNotFoundError("❌ Could not find Fashion-Gen H5 file.")

# Extract images from H5 file
print("📦 Extracting images from H5 file...")

# Create a temporary directory for extracted images
EXTRACTED_IMAGES_DIR = "/kaggle/working/extracted_fashion_images"
os.makedirs(EXTRACTED_IMAGES_DIR, exist_ok=True)

# Open H5 file and extract images
with h5py.File(H5_FILE_PATH, 'r') as h5_file:
    print(f"   H5 file keys: {list(h5_file.keys())}")
    
    # Common key names in Fashion-Gen dataset
    possible_image_keys = ['input_image', 'images', 'image', 'input_images', 'data']
    
    image_key = None
    for key in possible_image_keys:
        if key in h5_file:
            image_key = key
            break
    
    # If no common key found, try to find any dataset with image-like shape
    if image_key is None: 
        for key in h5_file.keys():
            dataset = h5_file[key]
            if isinstance(dataset, h5py.Dataset):
                shape = dataset.shape
                # Check if it looks like images (N, H, W, C) or (N, C, H, W)
                if len(shape) == 4 and (shape[-1] == 3 or shape[1] == 3):
                    image_key = key
                    print(f"   Found image dataset: {key} with shape {shape}")
                    break
    
    if image_key is None: 
        print("   Available keys and their shapes:")
        for key in h5_file.keys():
            if isinstance(h5_file[key], h5py. Dataset):
                print(f"   - {key}: {h5_file[key].shape}")
        raise KeyError("Could not find image data in H5 file.  Check the keys above.")
    
    images_dataset = h5_file[image_key]
    total_images = images_dataset.shape[0]
    print(f"   Total images in dataset: {total_images}")
    
    # Extract up to 100 images (evenly spaced for variety)
    num_to_extract = min(100, total_images)
    indices = np.linspace(0, total_images - 1, num_to_extract, dtype=int)
    
    for i, idx in enumerate(indices):
        img_array = images_dataset[idx]
        
        # Handle different formats (N, H, W, C) vs (N, C, H, W)
        if img_array.shape[0] == 3:  # (C, H, W) format
            img_array = np.transpose(img_array, (1, 2, 0))
        
        # Ensure uint8 format
        if img_array. max() <= 1.0:
            img_array = (img_array * 255).astype(np.uint8)
        else:
            img_array = img_array.astype(np.uint8)
        
        # Save image
        img = Image.fromarray(img_array)
        save_path = os. path.join(EXTRACTED_IMAGES_DIR, f"fashion_{idx:05d}.png")
        img.save(save_path)
        
        sample_image_paths[f"Image {i+1}: fashion_{idx:05d}"] = save_path
        
        if (i + 1) % 20 == 0:
            print(f"   Extracted {i + 1}/{num_to_extract} images...")

print(f"✅ Extracted {len(sample_image_paths)} images from Fashion-Gen dataset")

# --- 3. Load Pipeline ---
print("⚙️ Loading ControlNet...")
controlnet = ControlNetModel. from_pretrained(
    "lllyasviel/control_v11p_sd15_canny",
    torch_dtype=torch.float16
)

print("⚙️ Loading Stable Diffusion Pipeline...")
pipe = StableDiffusionControlNetPipeline. from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe. scheduler.config)

print("⚙️ Loading LoRA Weights...")
pipe.load_lora_weights(LORA_WEIGHTS_PATH, weight_name="pytorch_lora_weights.safetensors")

print("✅ Model loaded successfully!")

🔍 Searching for LoRA weights...
✅ Found LoRA weights at: /kaggle/input/fashion-lora-output
🔍 Searching for Fashion-Gen H5 dataset...
✅ Found Fashion-Gen H5 file:  /kaggle/input/fashiongen-validation/fashiongen_256_256_train.h5
📦 Extracting images from H5 file...
   H5 file keys: ['index', 'index_2', 'input_brand', 'input_category', 'input_composition', 'input_concat_description', 'input_department', 'input_description', 'input_gender', 'input_image', 'input_msrpUSD', 'input_name', 'input_pose', 'input_productID', 'input_season', 'input_subcategory']
   Total images in dataset: 260490
   Extracted 20/100 images...
   Extracted 40/100 images...
   Extracted 60/100 images...
   Extracted 80/100 images...
   Extracted 100/100 images...
✅ Extracted 100 images from Fashion-Gen dataset
⚙️ Loading ControlNet...
⚙️ Loading Stable Diffusion Pipeline...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


⚙️ Loading LoRA Weights...
✅ Model loaded successfully!


In [21]:
# CELL 3: Helper Functions
import random
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

def get_canny_edge(pil_image, low_threshold=100, high_threshold=200):
    """Extract Canny edges from an image."""
    img_array = np.array(pil_image)
    if len(img_array.shape) == 2:
        img_array = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
    edges = cv2.Canny(img_array, low_threshold, high_threshold)
    edges_rgb = np.stack([edges] * 3, axis=-1)
    return Image.fromarray(edges_rgb)

def generate_image(image, prompt, steps=20, guidance=7.5, seed=-1):
    """Generate a fashion image."""
    # Resize
    image = image.resize((256, 256), Image.LANCZOS)
    
    # Get edges
    edge_image = get_canny_edge(image)
    
    # Set seed
    if seed == -1:
        seed = random.randint(0, 2147483647)
    generator = torch.Generator(device="cuda").manual_seed(int(seed))
    
    # Generate
    with torch.inference_mode():
        output = pipe(
            prompt=prompt,
            image=edge_image,
            num_inference_steps=int(steps),
            guidance_scale=guidance,
            generator=generator
        )
    
    return edge_image, output. images[0], seed

def display_results(reference, edges, generated, prompt, seed):
    """Display results in a nice grid."""
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    
    axes[0].imshow(reference)
    axes[0].set_title("Reference Image")
    axes[0].axis('off')
    
    axes[1].imshow(edges)
    axes[1]. set_title("Edge Map")
    axes[1].axis('off')
    
    axes[2].imshow(generated)
    axes[2]. set_title("Generated")
    axes[2].axis('off')
    
    plt.suptitle(f'Prompt: "{prompt}"\nSeed:  {seed}', fontsize=10)
    plt.tight_layout()
    plt.show()

print("✅ Helper functions defined.")

✅ Helper functions defined.


In [22]:
# CELL 4: Enhanced Interactive Widget Interface (Fixed Random Samples)
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import random
import torch

# --- Preset Prompts (Organized by Category) ---
PRESET_PROMPTS = {
    "🧥 Outerwear": {
        "Black Leather Jacket": "A classic black leather jacket with silver zippers and fitted silhouette",
        "Navy Blazer": "A tailored navy blue blazer with gold buttons and sharp lapels",
        "Denim Jacket": "A vintage medium-wash denim jacket with brass buttons",
        "Bomber Jacket": "A brown leather bomber jacket with shearling collar",
        "Wool Overcoat": "An elegant charcoal wool overcoat with notched lapels",
    },
    "👔 Tops": {
        "White Linen Shirt": "A crisp white linen shirt with relaxed fit",
        "Silk Blouse": "An elegant cream silk blouse with subtle sheen",
        "Cashmere Sweater": "A cozy beige cashmere sweater with ribbed texture",
        "Striped T-Shirt": "A classic navy and white striped cotton t-shirt",
        "Wool Cardigan": "A chunky cream wool cardigan with wooden buttons",
    },
    "👗 Dresses": {
        "Red Silk Dress":  "A bright red silk evening dress with elegant draping",
        "Velvet Cocktail":  "A deep burgundy velvet cocktail dress",
        "Little Black Dress": "A sophisticated black fitted dress with clean lines",
        "Floral Sundress": "A light floral print sundress with flowing skirt",
    },
    "👖 Bottoms":  {
        "Dark Jeans": "Classic dark indigo straight-leg denim jeans",
        "Tailored Trousers": "Elegant black tailored wool trousers",
        "Leather Pants": "Sleek black leather pants with slim fit",
    }
}

# --- Custom CSS Styling ---
style_html = HTML("""
<style>
.widget-label { font-weight: bold ! important; }
. generator-title { 
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 20px;
    border-radius: 10px;
    margin-bottom: 20px;
    text-align: center;
}
.generator-title h1 { margin: 0; font-size: 28px; }
. generator-title p { margin: 10px 0 0 0; opacity: 0.9; }
. section-title { 
    color: #495057; 
    font-size: 16px; 
    font-weight: bold; 
    margin-bottom: 10px;
    border-bottom: 2px solid #667eea;
    padding-bottom: 5px;
}
. tip-box {
    background:  linear-gradient(135deg, #f5f7fa 0%, #e4e8eb 100%);
    border-left: 4px solid #667eea;
    padding:  15px;
    margin: 15px 0;
    border-radius:  0 8px 8px 0;
}
.results-title {
    background: #28a745;
    color: white;
    padding: 10px 20px;
    border-radius: 8px;
    display: inline-block;
    margin:  20px 0 10px 0;
}
. status-success { color: #28a745; font-weight: bold; }
.status-error { color: #dc3545; font-weight: bold; }
.status-loading { color: #ffc107; font-weight: bold; }
</style>
""")
display(style_html)

# --- Title ---
title = widgets.HTML("""
<div class="generator-title">
    <h1>👗 Fashion Image Generator</h1>
    <p>Transform reference images into stunning fashion designs using AI</p>
</div>
""")

# ==================== SAMPLE IMAGE MANAGEMENT ====================

# Store ALL sample paths globally
ALL_SAMPLE_PATHS = dict(sample_image_paths)  # Make a copy
ALL_SAMPLE_KEYS = list(ALL_SAMPLE_PATHS.keys())

print(f"📁 Total available images: {len(ALL_SAMPLE_KEYS)}")

def get_diverse_samples(n=10):
    """Select diverse samples by picking every nth image."""
    if len(ALL_SAMPLE_KEYS) <= n:
        return ALL_SAMPLE_KEYS. copy()
    step = max(1, len(ALL_SAMPLE_KEYS) // n)
    return [ALL_SAMPLE_KEYS[i * step] for i in range(min(n, len(ALL_SAMPLE_KEYS)))]

def get_random_samples(n=10):
    """Get completely random samples."""
    return random.sample(ALL_SAMPLE_KEYS, min(n, len(ALL_SAMPLE_KEYS)))

# Initialize with diverse samples
current_samples = get_diverse_samples(10)

# ==================== WIDGETS ====================

# --- Image Selection ---
sample_dropdown = widgets. Dropdown(
    options=current_samples,
    value=current_samples[0] if current_samples else None,
    description='',
    layout=widgets.Layout(width='100%')
)

image_preview = widgets.Output(layout=widgets.Layout(
    width='100%', 
    min_height='220px',
    display='flex',
    justify_content='center',
    align_items='center'
))

prev_btn = widgets. Button(description='◀ Prev', layout=widgets.Layout(width='80px'))
next_btn = widgets.Button(description='Next ▶', layout=widgets.Layout(width='80px'))
random_img_btn = widgets. Button(description='🎲 Random', button_style='info', layout=widgets. Layout(width='100px'))
shuffle_btn = widgets. Button(description='🔀 New Set', button_style='warning', layout=widgets. Layout(width='100px'))

# Image counter display
image_counter = widgets.HTML(value=f'<small style="color:#6c757d">Showing 10 of {len(ALL_SAMPLE_KEYS)} images</small>')

# --- Category and Prompt Selection ---
category_dropdown = widgets. Dropdown(
    options=list(PRESET_PROMPTS.keys()),
    value=list(PRESET_PROMPTS.keys())[0],
    description='',
    layout=widgets.Layout(width='100%')
)

prompt_dropdown = widgets.Dropdown(
    options=list(PRESET_PROMPTS[list(PRESET_PROMPTS.keys())[0]].keys()),
    description='',
    layout=widgets.Layout(width='100%')
)

prompt_input = widgets. Textarea(
    value=list(PRESET_PROMPTS[list(PRESET_PROMPTS.keys())[0]].values())[0],
    placeholder='Describe your garment in detail...',
    layout=widgets.Layout(width='100%', height='80px')
)

random_prompt_btn = widgets.Button(
    description='🎲 Surprise Me! ', 
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

# --- Settings ---
steps_slider = widgets. IntSlider(
    value=20, min=10, max=40, step=5,
    description='',
    layout=widgets.Layout(width='100%'),
    style={'handle_color': '#667eea'}
)

guidance_slider = widgets.FloatSlider(
    value=7.5, min=1.0, max=12.0, step=0.5,
    description='',
    layout=widgets. Layout(width='100%'),
    style={'handle_color':  '#667eea'}
)

seed_input = widgets.IntText(
    value=-1,
    layout=widgets.Layout(width='120px')
)

# --- Generation Mode ---
mode_toggle = widgets. ToggleButtons(
    options=['🎨 Single', '📦 Batch (Category)', '🎯 Batch (Custom)'],
    value='🎨 Single',
    layout=widgets.Layout(width='100%')
)

# Batch custom prompts input
batch_prompts_input = widgets.Textarea(
    value="A black leather jacket\nA white silk blouse\nA red evening dress\nA navy blazer",
    placeholder='Enter one prompt per line (max 6)...',
    layout=widgets.Layout(width='100%', height='120px')
)

# --- Action Buttons ---
generate_btn = widgets. Button(
    description='🚀 Generate',
    button_style='success',
    layout=widgets.Layout(width='200px', height='50px')
)

clear_btn = widgets.Button(
    description='🗑️ Clear Results',
    button_style='danger',
    layout=widgets. Layout(width='150px', height='40px')
)

# --- Output Areas ---
status_output = widgets.Output(layout=widgets.Layout(width='100%', min_height='30px'))
results_output = widgets.Output(layout=widgets. Layout(width='100%'))

# ==================== EVENT HANDLERS ====================

def update_image_preview(*args):
    """Update the image preview."""
    with image_preview: 
        clear_output(wait=True)
        sample_name = sample_dropdown. value
        
        # Use ALL_SAMPLE_PATHS to get the actual path
        if sample_name and sample_name in ALL_SAMPLE_PATHS:
            try:
                img = Image.open(ALL_SAMPLE_PATHS[sample_name]).convert("RGB")
                
                fig, ax = plt.subplots(1, 1, figsize=(3, 3))
                ax.imshow(img)
                ax.axis('off')
                ax.set_title(sample_name, fontsize=10, fontweight='bold')
                plt.tight_layout()
                plt.show()
                plt.close()
            except Exception as e:
                print(f"Error loading image: {e}")
        else:
            print("No image selected")

def on_prev_image(b):
    """Navigate to previous image."""
    options = list(sample_dropdown.options)
    if not options:
        return
    current_idx = options. index(sample_dropdown.value) if sample_dropdown. value in options else 0
    new_idx = (current_idx - 1) % len(options)
    sample_dropdown.value = options[new_idx]

def on_next_image(b):
    """Navigate to next image."""
    options = list(sample_dropdown.options)
    if not options: 
        return
    current_idx = options.index(sample_dropdown.value) if sample_dropdown.value in options else 0
    new_idx = (current_idx + 1) % len(options)
    sample_dropdown.value = options[new_idx]

def on_random_image(b):
    """Select random image from current set."""
    options = list(sample_dropdown.options)
    if options:
        sample_dropdown.value = random.choice(options)

def on_shuffle_samples(b):
    """Shuffle to get a completely new random set of samples."""
    global current_samples
    
    # Get new random samples from ALL available images
    new_samples = get_random_samples(10)
    current_samples = new_samples
    
    # Update dropdown options
    sample_dropdown. options = new_samples
    sample_dropdown.value = new_samples[0]
    
    # Update counter
    image_counter.value = f'<small style="color:#6c757d">Showing 10 of {len(ALL_SAMPLE_KEYS)} images (shuffled! )</small>'
    
    with status_output:
        clear_output(wait=True)
        display(HTML('<p style="color:#28a745; font-weight: bold;">🔀 Loaded new random set of 10 reference images! </p>'))

def update_prompt_options(change):
    """Update prompt dropdown when category changes."""
    category = change['new']
    prompts = list(PRESET_PROMPTS[category].keys())
    prompt_dropdown.options = prompts
    prompt_dropdown.value = prompts[0]
    prompt_input.value = PRESET_PROMPTS[category][prompts[0]]

def update_prompt_text(change):
    """Update prompt text when preset changes."""
    category = category_dropdown.value
    prompt_name = change['new']
    if prompt_name in PRESET_PROMPTS[category]:
        prompt_input.value = PRESET_PROMPTS[category][prompt_name]

def on_random_prompt(b):
    """Select random category and prompt."""
    category = random.choice(list(PRESET_PROMPTS.keys()))
    category_dropdown.value = category
    prompt_name = random.choice(list(PRESET_PROMPTS[category].keys()))
    prompt_dropdown.value = prompt_name

def on_clear_results(b):
    """Clear the results area."""
    with results_output: 
        clear_output()
    with status_output: 
        clear_output()
        print("🧹 Results cleared!")

def show_status(message, status_type="loading"):
    """Display status message with styling."""
    with status_output: 
        clear_output(wait=True)
        if status_type == "loading":
            display(HTML(f'<p class="status-loading">⏳ {message}</p>'))
        elif status_type == "success":
            display(HTML(f'<p class="status-success">✅ {message}</p>'))
        elif status_type == "error":
            display(HTML(f'<p class="status-error">❌ {message}</p>'))

def display_single_result(reference, edges, generated, prompt, seed):
    """Display single generation result beautifully."""
    fig = plt.figure(figsize=(14, 5))
    gs = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 0.1, 1.2])
    
    # Reference
    ax1 = fig.add_subplot(gs[0])
    ax1.imshow(reference)
    ax1.set_title('📷 Reference', fontsize=12, fontweight='bold', color='#495057')
    ax1.axis('off')
    
    # Edge Map
    ax2 = fig.add_subplot(gs[1])
    ax2.imshow(edges)
    ax2.set_title('🔲 Structure', fontsize=12, fontweight='bold', color='#495057')
    ax2.axis('off')
    
    # Arrow
    ax_arrow = fig.add_subplot(gs[2])
    ax_arrow.annotate('', xy=(0.9, 0.5), xytext=(0.1, 0.5),
                      arrowprops=dict(arrowstyle='->', color='#667eea', lw=3))
    ax_arrow.axis('off')
    
    # Generated
    ax3 = fig.add_subplot(gs[3])
    ax3.imshow(generated)
    ax3.set_title('✨ Generated', fontsize=12, fontweight='bold', color='#28a745')
    ax3.axis('off')
    
    # Add prompt as subtitle
    display_prompt = f'"{prompt[: 80]}..."' if len(prompt) > 80 else f'"{prompt}"'
    fig.suptitle(display_prompt, fontsize=10, style='italic', color='#6c757d', y=0.02)
    
    plt.tight_layout()
    
    filename = f"generated_seed{seed}.png"
    plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white', edgecolor='none')
    plt.show()
    plt.close()
    
    display(HTML(f'''
        <div style="background:#d4edda; padding:10px; border-radius:5px; margin-top:10px; text-align:center;">
            <b>💾 Saved: </b> {filename} | <b>🎲 Seed:</b> {seed}
        </div>
    '''))

def display_batch_results(reference, edges, results, title_text="Batch Generation"):
    """Display batch generation results in a beautiful grid."""
    n = len(results)
    
    if n == 0:
        print("No results to display!")
        return
    
    # Calculate layout
    cols = min(3, n)
    rows = (n + cols - 1) // cols
    
    fig = plt.figure(figsize=(16, 3 + rows * 4))
    
    # Top row: Reference and Edge
    gs_top = gridspec.GridSpec(1, 2, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.68)
    
    ax_ref = fig. add_subplot(gs_top[0])
    ax_ref.imshow(reference)
    ax_ref.set_title('📷 Reference Image', fontsize=12, fontweight='bold')
    ax_ref.axis('off')
    
    ax_edge = fig. add_subplot(gs_top[1])
    ax_edge.imshow(edges)
    ax_edge.set_title('🔲 Extracted Structure', fontsize=12, fontweight='bold')
    ax_edge.axis('off')
    
    # Generated images grid
    gs_bottom = gridspec.GridSpec(rows, cols, figure=fig, left=0.05, right=0.95, 
                                   top=0.62, bottom=0.02, hspace=0.3, wspace=0.1)
    
    for i, (img, prompt) in enumerate(results):
        row = i // cols
        col = i % cols
        ax = fig.add_subplot(gs_bottom[row, col])
        ax.imshow(img)
        
        short_prompt = prompt[: 30] + "..." if len(prompt) > 30 else prompt
        ax.set_title(short_prompt, fontsize=9, fontweight='bold', color='#495057', pad=5)
        ax.axis('off')
        
        for spine in ax.spines.values():
            spine.set_visible(True)
            spine.set_color('#667eea')
            spine.set_linewidth(2)
    
    plt. suptitle(title_text, fontsize=14, fontweight='bold', y=0.98)
    
    filename = "batch_generation.png"
    plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white', edgecolor='none')
    plt.show()
    plt.close()
    
    display(HTML(f'''
        <div style="background:#d4edda; padding:10px; border-radius:5px; margin-top: 10px; text-align:center;">
            <b>💾 Saved:</b> {filename} | <b>🎯 Generated:</b> {n} images
        </div>
    '''))

def on_generate(b):
    """Handle generation based on selected mode."""
    mode = mode_toggle.value
    
    # Get reference image - USE ALL_SAMPLE_PATHS
    sample_name = sample_dropdown.value
    if sample_name not in ALL_SAMPLE_PATHS: 
        show_status("Please select a reference image!", "error")
        return
    
    ref_image = Image.open(ALL_SAMPLE_PATHS[sample_name]).convert("RGB")
    ref_image = ref_image.resize((256, 256), Image.LANCZOS)
    edge_image = get_canny_edge(ref_image)
    
    steps = steps_slider. value
    guidance = guidance_slider. value
    seed = seed_input. value
    
    with results_output:
        clear_output(wait=True)
        
        if mode == '🎨 Single':
            prompt = prompt_input. value. strip()
            if not prompt:
                show_status("Please enter a prompt!", "error")
                return
            
            show_status(f"Generating:  {prompt[:50]}...")
            
            try:
                if seed == -1:
                    seed = random. randint(0, 2147483647)
                generator = torch.Generator(device="cuda").manual_seed(int(seed))
                
                with torch.inference_mode():
                    output = pipe(
                        prompt=prompt,
                        image=edge_image,
                        num_inference_steps=steps,
                        guidance_scale=guidance,
                        generator=generator
                    )
                
                display_single_result(ref_image, edge_image, output. images[0], prompt, seed)
                show_status(f"Generation complete!  Seed: {seed}", "success")
                
            except Exception as e:
                show_status(f"Error:  {str(e)}", "error")
        
        elif mode == '📦 Batch (Category)':
            category = category_dropdown.value
            prompts = list(PRESET_PROMPTS[category].items())[:6]
            
            show_status(f"Generating {len(prompts)} images from '{category}'...")
            
            try:
                results = []
                for i, (name, prompt) in enumerate(prompts):
                    show_status(f"Generating {i+1}/{len(prompts)}: {name}...")
                    
                    gen_seed = seed + i if seed != -1 else random.randint(0, 2147483647)
                    generator = torch.Generator(device="cuda").manual_seed(int(gen_seed))
                    
                    with torch.inference_mode():
                        output = pipe(
                            prompt=prompt,
                            image=edge_image,
                            num_inference_steps=steps,
                            guidance_scale=guidance,
                            generator=generator
                        )
                    
                    results.append((output.images[0], name))
                
                display_batch_results(ref_image, edge_image, results, f"Batch:  {category}")
                show_status(f"Batch complete!  Generated {len(results)} images.", "success")
                
            except Exception as e:
                show_status(f"Error: {str(e)}", "error")
        
        elif mode == '🎯 Batch (Custom)':
            prompts = [p. strip() for p in batch_prompts_input.value.strip().split('\n') if p.strip()]
            prompts = prompts[:6]
            
            if not prompts: 
                show_status("Please enter at least one prompt!", "error")
                return
            
            show_status(f"Generating {len(prompts)} custom images...")
            
            try:
                results = []
                for i, prompt in enumerate(prompts):
                    show_status(f"Generating {i+1}/{len(prompts)}: {prompt[:30]}...")
                    
                    gen_seed = seed + i if seed != -1 else random.randint(0, 2147483647)
                    generator = torch.Generator(device="cuda").manual_seed(int(gen_seed))
                    
                    with torch.inference_mode():
                        output = pipe(
                            prompt=prompt,
                            image=edge_image,
                            num_inference_steps=steps,
                            guidance_scale=guidance,
                            generator=generator
                        )
                    
                    results.append((output.images[0], prompt))
                
                display_batch_results(ref_image, edge_image, results, "Batch: Custom Prompts")
                show_status(f"Batch complete!  Generated {len(results)} images.", "success")
                
            except Exception as e: 
                show_status(f"Error:  {str(e)}", "error")

# ==================== CONNECT EVENTS ====================

sample_dropdown.observe(update_image_preview, names='value')
category_dropdown.observe(update_prompt_options, names='value')
prompt_dropdown.observe(update_prompt_text, names='value')

prev_btn.on_click(on_prev_image)
next_btn.on_click(on_next_image)
random_img_btn. on_click(on_random_image)
shuffle_btn.on_click(on_shuffle_samples)
random_prompt_btn. on_click(on_random_prompt)
generate_btn.on_click(on_generate)
clear_btn.on_click(on_clear_results)

# Initialize preview
update_image_preview()

# ==================== LAYOUT ====================

# Image Selection Section
image_section = widgets.VBox([
    widgets.HTML('<div class="section-title">📷 Reference Image</div>'),
    sample_dropdown,
    widgets.HBox([prev_btn, random_img_btn, next_btn], 
                 layout=widgets.Layout(justify_content='center', margin='5px 0')),
    widgets.HBox([shuffle_btn], 
                 layout=widgets.Layout(justify_content='center', margin='5px 0')),
    image_counter,
    image_preview,
], layout=widgets.Layout(width='300px', padding='15px', 
                          border='1px solid #dee2e6', border_radius='10px',
                          background_color='#f8f9fa'))

# Prompt Section
prompt_section = widgets.VBox([
    widgets.HTML('<div class="section-title">✏️ Prompt</div>'),
    widgets.HTML('<small style="color:#6c757d">Category: </small>'),
    category_dropdown,
    widgets.HTML('<small style="color:#6c757d">Preset:</small>'),
    widgets.HBox([prompt_dropdown, random_prompt_btn]),
    widgets.HTML('<small style="color:#6c757d">Or write your own: </small>'),
    prompt_input,
], layout=widgets.Layout(width='400px', padding='15px',
                          border='1px solid #dee2e6', border_radius='10px',
                          background_color='#f8f9fa'))

# Settings Section
settings_section = widgets.VBox([
    widgets. HTML('<div class="section-title">⚙️ Settings</div>'),
    widgets.HTML('<small style="color:#6c757d">Quality Steps (10-40):</small>'),
    steps_slider,
    widgets. HTML('<small style="color:#6c757d">Prompt Strength (1-12):</small>'),
    guidance_slider,
    widgets. HBox([
        widgets.HTML('<small style="color:#6c757d; margin-right: 10px">Seed:</small>'),
        seed_input,
        widgets.HTML('<small style="color:#999; margin-left:10px">(-1 = random)</small>')
    ]),
], layout=widgets.Layout(width='280px', padding='15px',
                          border='1px solid #dee2e6', border_radius='10px',
                          background_color='#f8f9fa'))

# Mode & Batch Section
mode_section = widgets.VBox([
    widgets.HTML('<div class="section-title">🎬 Generation Mode</div>'),
    mode_toggle,
    widgets.HTML('<div style="margin-top:10px"><small style="color:#6c757d">Custom batch prompts (one per line, max 6):</small></div>'),
    batch_prompts_input,
], layout=widgets.Layout(padding='15px', margin='10px 0',
                          border='1px solid #dee2e6', border_radius='10px',
                          background_color='#f8f9fa'))

# Action Section
action_section = widgets.VBox([
    widgets.HBox([generate_btn, clear_btn], 
                 layout=widgets.Layout(justify_content='center', gap='20px')),
    status_output,
], layout=widgets.Layout(padding='15px', margin='10px 0'))

# Tips
tips_section = widgets.HTML("""
<div class="tip-box">
    <b>💡 Pro Tips:</b><br>
    • <b>🔀 New Set:</b> Get a completely new random set of reference images from all available samples<br>
    • <b>🎲 Random:</b> Pick a random image from the current set of 10<br>
    • <b>Single Mode:</b> Generate one image with full control<br>
    • <b>Batch (Category):</b> Generate all presets from selected category<br>
    • <b>Batch (Custom):</b> Enter your own prompts, one per line (max 6)
</div>
""")

# Results Section Header
results_header = widgets.HTML('<div class="results-title">🖼️ Generated Results</div>')

# ==================== DISPLAY ====================

# Main Layout
top_row = widgets.HBox([image_section, prompt_section, settings_section],
                       layout=widgets.Layout(gap='15px', justify_content='center'))

display(title)
display(top_row)
display(mode_section)
display(action_section)
display(tips_section)
display(results_header)
display(results_output)

print("✅ Enhanced UI loaded!  Select an image, choose a prompt, and click Generate!")

📁 Total available images: 100


HTML(value='\n<div class="generator-title">\n    <h1>👗 Fashion Image Generator</h1>\n    <p>Transform referenc…

HTML(value='\n<div class="tip-box">\n    <b>💡 Pro Tips:</b><br>\n    • <b>🔀 New Set:</b> Get a completely new …

HTML(value='<div class="results-title">🖼️ Generated Results</div>')

Output(layout=Layout(width='100%'))

✅ Enhanced UI loaded!  Select an image, choose a prompt, and click Generate!
